In [136]:
import asyncio
import unidecode
import re
from selenium import webdriver
from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import pygsheets
import os
from fuzzywuzzy import process


In [137]:

gc = pygsheets.authorize(client_secret='/Users/acapai/Documents/Git/Espace-test/Scrapping-With-Python/Scrapping_Url_GAFEO/Oauth_gg/code_secret_client_95743482524-gj2mnoav9naiqt454ggvt71r28r4n3dk.apps.googleusercontent.com.json')

# Suivi Eron 2022 (derniere version)
#sh = gc.open_by_key('13VqSH8KjAzB3-mroVhtUJjXgO2Gs31UtpqdehiLMyRs')

# DUPLICATA Suivi Eron 2022 (derniere version)
sh = gc.open_by_key('1Ix4xc_kJPrIBXQL8JmGVz_XNnY4t7AMHdHvmyVlExiA')

wksheet_depart_inf_01_01 = sh[4]
col_name_apprenant = wksheet_depart_inf_01_01.get_col(9)
col_formation_raw = wksheet_depart_inf_01_01.get_col(8)

# print(col_name_apprenant)
print(wksheet_depart_inf_01_01.title)
print(sh.title)

col_apprenant = [string for string in col_name_apprenant if string ]
col_formation = [ string.split("-")[0] for string in col_formation_raw if string]


['lien Dolibar', '', '', '', '', 'Nom Prénom', 'FARFART Alexia', 'ANSELIN Cindy', 'HUE Marielle Dominique Angelique', 'DOLADILLE Sandrine', 'LECHERBONNIER Aurelie Gladys Alienor', 'BRABANT Florine Sophie Francine', 'ARSENE Carole Michele Isabelle', 'DAVID Stéphanie Marie Anne', 'BRACIKOWSKI Elisabeth', 'BELCARZ Delphine', 'VANPEPERSTRAETE Béatrice Jacqueline', 'BAUDOIN Harmony Veronique Stephanie', 'BAUDOIN Harmony Veronique Stephanie', 'CHAPTARD Valerie', 'MILLE Isabelle Jacqueline Antoinette', 'BRAYE Kevin', 'HOHL Sabine', 'BONVALET Emilie Ginette Annette', 'CAZAUX Delphine Laurence', 'CAPITAINE Florie', 'DA CUNHA Sophie', 'CAPPOEN Gwenaelle Therese Marie', 'HEROUT Sandrine Nathalie Severine', 'DAGON Estelle', 'FOISSOTE Celine Lydia Denise', 'LABAT Melanie', 'JAMMES Marina Laure Emeline', 'DUONG Jennifer Marine', 'CHESNEAU Maeva Claudine', 'LALANNE Eloise Marie', 'RIPOCHE Marie', 'SICARDI Virginie Josette', 'DERVILLERS Vivien Freddy', 'DELMOTTE Karine Raymonde Georgette', 'WISSART Al

In [138]:
col_formation_raw = wksheet_depart_inf_01_01.get_col(8)
col_formation = [ string.split("-")[0] for string in col_formation_raw if string]


In [179]:
datasheet = pd.DataFrame(list(zip(col_formation[2:],col_apprenant[2:])),columns=['Formation','Apprenant'])
datasheet['Dernier Accès'] = ''
datasheet['Pourcentage']=''
datasheet['Apprenant_Normalize']=''
datasheet["Apprenant_Gafeo"]=""


In [180]:
#Normalize name 
for index,row in datasheet.iterrows():
    print(unidecode.unidecode(row['Apprenant']))
    print(row['Apprenant'].lower())
    print(unidecode.unidecode(row['Apprenant'].lower()))
    print(re.sub(r'[^a-zA-Z0-9]','',unidecode.unidecode(row['Apprenant'].lower().replace(' ', ''))))
    # row['Apprenant_Normalize']= re.sub(r'[^a-zA-Z0-9]','',unidecode.unidecode(row['Apprenant']).lower().replace(' ', ''))
    row['Apprenant_Normalize']= unidecode.unidecode(row['Apprenant']).lower()

FARFART Alexia
farfart alexia
farfart alexia
farfartalexia
ANSELIN Cindy
anselin cindy
anselin cindy
anselincindy
HUE Marielle Dominique Angelique
hue marielle dominique angelique
hue marielle dominique angelique
huemarielledominiqueangelique
DOLADILLE Sandrine
doladille sandrine
doladille sandrine
doladillesandrine
LECHERBONNIER Aurelie Gladys Alienor
lecherbonnier aurelie gladys alienor
lecherbonnier aurelie gladys alienor
lecherbonnieraureliegladysalienor
BRABANT Florine Sophie Francine
brabant florine sophie francine
brabant florine sophie francine
brabantflorinesophiefrancine
ARSENE Carole Michele Isabelle
arsene carole michele isabelle
arsene carole michele isabelle
arsenecarolemicheleisabelle
DAVID Stephanie Marie Anne
david stéphanie marie anne
david stephanie marie anne
davidstephaniemarieanne
BRACIKOWSKI Elisabeth
bracikowski elisabeth
bracikowski elisabeth
bracikowskielisabeth
BELCARZ Delphine
belcarz delphine
belcarz delphine
belcarzdelphine
VANPEPERSTRAETE Beatrice Jacquel

In [181]:
formation_unique= datasheet.Formation.unique().tolist()


In [182]:
dic_dataframe={}
for formation in formation_unique:
    dic_dataframe[formation]=datasheet[datasheet["Formation"]==formation]
print(dic_dataframe[formation_unique[-2]])

   Formation                           Apprenant Dernier Accès Pourcentage  \
19    INF055                    CAPITAINE Florie                             
21    INF055     CAPPOEN Gwenaelle Therese Marie                             
27    INF055               DUONG Jennifer Marine                             
33    INF055  DELMOTTE Karine Raymonde Georgette                             
48    INF055   BECART Séverine Claudine Michelle                             

                   Apprenant_Normalize Apprenant_Gafeo  
19                    capitaine florie                  
21     cappoen gwenaelle therese marie                  
27               duong jennifer marine                  
33  delmotte karine raymonde georgette                  
48   becart severine claudine michelle                  


In [183]:
formation_key_dic=list(dic_dataframe.keys())
print(formation_key_dic)

['INF057', 'INF058', 'INF025', 'INF002', 'INF003', 'INF014', 'INF055', 'INF056']


In [149]:
if os.path.exists("/Applications/Internet/Google Chrome.app/Contents/MacOS/Google Chrome"):
        path_google_chrome="/Applications/Internet/Google Chrome.app/Contents/MacOS/Google Chrome"
else:
    path_google_chrome="/Applications/Google Chrome.app/Contents/MacOS/Google Chrome"
options = Options()
options.binary_location = path_google_chrome
options.add_experimental_option("detach", True)
# options.add_argument("--headless")
# options.add_argument("--disable-gpu")
# options.add_experimental_option('prefs', {'download.default_directory':'/Users/acapai/Downloads/'})
# options.add_experimental_option('prefs',{"profile.default_content_setting_values.automatic_downloads": 1})

rel_path="/Applications/chromedriver"
PATH = os.path.abspath(rel_path)

id="andria.c@eronservice.fr"
psswd="Andria-2021!"
browser = webdriver.Chrome(options=options,executable_path=PATH)
browser.get("https://www.gafeo.fr/my/")
browser.find_element(By.ID,'username').send_keys(id)
browser.find_element(By.ID,'password').send_keys(psswd)
browser.find_element(By.CSS_SELECTOR,"button.btn-primary").click()
div_searchbox = WebDriverWait(browser,10).until(EC.presence_of_element_located((By.ID,"coursesearchbox")))

/var/folders/9g/9qptqtl550vchpg029w9bzjr0000gn/T/ipykernel_18088/1014642302.py:18: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(options=options,executable_path=PATH)


In [148]:
# POUR RETOURNER EN ARRIERE
browser.execute_script("window.history.go(-1)")

In [150]:
inputs_field = WebDriverWait(browser,10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR,"#coursesearch .coursesearchbox input")))

In [151]:
inputs_field[0].send_keys(formation_key_dic[0])
inputs_field[1].click()

In [152]:
action_link = WebDriverWait(browser,10).until(EC.presence_of_all_elements_located((By.CLASS_NAME,"action-link")))
action_link[0].click()

In [153]:
div_button_config_course = WebDriverWait(browser,10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR,".teacherdash a")))
div_button_config_course[0].click()

In [154]:
WebDriverWait(browser,10).until(EC.presence_of_element_located((By.LINK_TEXT,"Rapports de session"))).click()

In [155]:
dp_down_list_participant = Select(WebDriverWait(browser,10).until(EC.presence_of_element_located((By.CSS_SELECTOR,"select[name='userid']"))))

In [156]:
select_box = browser.find_element_by_name("userid") 
options = [x for x in select_box.find_elements(By.TAG_NAME,"option")]

/var/folders/9g/9qptqtl550vchpg029w9bzjr0000gn/T/ipykernel_18088/3432941428.py:1: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  select_box = browser.find_element_by_name("userid")


In [ ]:
for i,element in enumerate(options):
    print(element.get_attribute("value"))
    print(element.get_attribute("text"))
    # element_normalize=re.sub(r'[^a-zA-Z0-9]','',unidecode.unidecode(element.get_attribute("text")).lower().replace(' ', ''))
    element_normalize=re.sub(r'[^a-zA-Z0-9]','',unidecode.unidecode(element.get_attribute("text")).lower())
    list_apprenant= dic_dataframe[formation_key_dic[0]]['Apprenant_Normalize'].tolist()
    print("element_normalize:",element_normalize)
    print("list_apprenant:",list_apprenant)
    Ratios = process.extract(element_normalize,list_apprenant)
    print(Ratios)
    highest = process.extractOne(element_normalize,list_apprenant)
    print(highest)
    if element_normalize in dic_dataframe[formation_key_dic[0]]["Apprenant_Normalize"].tolist():
        print("YES")
    # print("dic_dataframe[formation_key_dic[0]]['Apprenant_Normalize']",dic_dataframe[formation_key_dic[0]]["Apprenant_Normalize"])
    # if element_normalize in dic_dataframe[formation_key_dic[0]]["Apprenant_Normalize"]:
        print("dic_dataframe[formation_key_dic[0]]['Apprenant_Normalize']==element_normalize",dic_dataframe[formation_key_dic[0]]["Apprenant_Normalize"]==element_normalize)
        dic_dataframe[formation_key_dic[0]].loc[dic_dataframe[formation_key_dic[0]]["Apprenant_Normalize"]==element_normalize,"Apprenant_Gafeo"]=element.get_attribute("text")

In [185]:
pd_copy = dic_dataframe[formation_key_dic[0]]
pd_copy["Ratio_Fuzzy"]=""
pd_copy["Name_Fuzzy"]=""
pd_copy["option_value"]=""
list_apprenant= pd_copy['Apprenant_Normalize'].tolist()
for i,element in enumerate(options):
    print(element.get_attribute("value"))
    print(element.get_attribute("text"))
    # element_normalize=re.sub(r'[^a-zA-Z0-9]','',unidecode.unidecode(element.get_attribute("text")).lower().replace(' ', ''))
    element_normalize=unidecode.unidecode(element.get_attribute("text")).lower()
    print("element_normalize:",element_normalize)
    print("list_apprenant:",list_apprenant)
    Ratios = process.extract(element_normalize,list_apprenant,limit=len(list_apprenant))
    print(Ratios)
    highest = process.extractOne(element_normalize,list_apprenant)
    print("Highest",highest)
    index= [x for x, y in enumerate(list_apprenant) if y == highest[0]]
    if highest[1]>75:
        print("Ratio",highest[1])
        print("index",index[0])
        pd_copy.iloc[index[0],pd_copy.columns.get_loc("Ratio_Fuzzy")]=highest[1]
        pd_copy.iloc[index[0],pd_copy.columns.get_loc("Name_Fuzzy")]=highest[0]
        pd_copy.iloc[index[0],pd_copy.columns.get_loc("option_value")]=element.get_attribute("value")
        pd_copy.iloc[index[0],pd_copy.columns.get_loc("Apprenant_Gafeo")]=element.get_attribute("text")

    

/var/folders/9g/9qptqtl550vchpg029w9bzjr0000gn/T/ipykernel_18088/1604669305.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pd_copy["Ratio_Fuzzy"]=""
/var/folders/9g/9qptqtl550vchpg029w9bzjr0000gn/T/ipykernel_18088/1604669305.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pd_copy["Name_Fuzzy"]=""
/var/folders/9g/9qptqtl550vchpg029w9bzjr0000gn/T/ipykernel_18088/1604669305.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_

9199
AMBERT Jean-Sebastien Pierre Charles
element_normalize: ambert jean-sebastien pierre charles
list_apprenant: ['farfart alexia', 'anselin cindy', 'mille isabelle jacqueline antoinette', 'braye kevin', 'cazaux delphine laurence', 'da cunha sophie', 'jammes marina laure emeline', 'sicardi virginie josette', 'wissart aline helene', 'gaehnke stacie isabelle']
[('anselin cindy', 49), ('da cunha sophie', 45), ('farfart alexia', 39), ('braye kevin', 38), ('jammes marina laure emeline', 36), ('wissart aline helene', 36), ('gaehnke stacie isabelle', 35), ('mille isabelle jacqueline antoinette', 31), ('sicardi virginie josette', 31), ('cazaux delphine laurence', 30)]
Highest ('anselin cindy', 49)
8798
Andria Capai
element_normalize: andria capai
list_apprenant: ['farfart alexia', 'anselin cindy', 'mille isabelle jacqueline antoinette', 'braye kevin', 'cazaux delphine laurence', 'da cunha sophie', 'jammes marina laure emeline', 'sicardi virginie josette', 'wissart aline helene', 'gaehnke stac